In [1]:

cd Fk-Diffusion-Steering/text_to_image

/home/safernandez/Fk-Diffusion-Steering/text_to_image


In [ ]:

from fastapi import FastAPI
from typing import List
from pydantic import BaseModel
from pyngrok import ngrok
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images
from transformers import AutoModelForCausalLM
from PIL import Image
import uuid, os
import nest_asyncio
import uvicorn
from fastapi.staticfiles import StaticFiles
import os
import sys
sys.path.append('fkd_diffusers')
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from pyngrok import ngrok
import torch, argparse, uuid, os
from datetime import datetime
from fks_utils import get_model
from diffusers import LCMScheduler, AutoPipelineForText2Image, StableDiffusionPipeline
from PIL import Image
from io import BytesIO
from typing import List, Optional
import torch
import requests
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from io import BytesIO
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from torchvision import transforms as tfms
from diffusers import StableDiffusionPipeline, DDIMScheduler


# Useful function for later
def load_image(url, size=None):
    response = requests.get(url, timeout=0.2)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    if size is not None:
        img = img.resize(size)
    return img

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
# Set up a DDIM scheduler

# Sample function (regular DDIM)
@torch.no_grad()
def sample(
    prompt,
    start_step=0,
    start_latents=None,
    guidance_scale=3.5,
    num_inference_steps=30,
    num_images_per_prompt=1,
    do_classifier_free_guidance=True,
    negative_prompt="",
    device=device,
):

    # Encode prompt
    text_embeddings = pipe._encode_prompt(
        prompt, device, num_images_per_prompt, do_classifier_free_guidance, negative_prompt
    )

    # Set num inference steps
    pipe.scheduler.set_timesteps(num_inference_steps, device=device)

    # Create a random starting point if we don't have one already
    if start_latents is None:
        start_latents = torch.randn(1, 4, 64, 64, device=device)
        start_latents *= pipe.scheduler.init_noise_sigma

    latents = start_latents.clone()

    for i in tqdm(range(start_step, num_inference_steps)):

        t = pipe.scheduler.timesteps[i]

        # Expand the latents if we are doing classifier free guidance
        latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
        latent_model_input = pipe.scheduler.scale_model_input(latent_model_input, t)

        # Predict the noise residual
        noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

        # Perform guidance
        if do_classifier_free_guidance:
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

        # Normally we'd rely on the scheduler to handle the update step:
        # latents = pipe.scheduler.step(noise_pred, t, latents).prev_sample

        # Instead, let's do it ourselves:
        prev_t = max(1, t.item() - (1000 // num_inference_steps))  # t-1
        alpha_t = pipe.scheduler.alphas_cumprod[t.item()]
        alpha_t_prev = pipe.scheduler.alphas_cumprod[prev_t]
        predicted_x0 = (latents - (1 - alpha_t).sqrt() * noise_pred) / alpha_t.sqrt()
        direction_pointing_to_xt = (1 - alpha_t_prev).sqrt() * noise_pred
        latents = alpha_t_prev.sqrt() * predicted_x0 + direction_pointing_to_xt

    # Post-processing
    images = pipe.decode_latents(latents)
    images = pipe.numpy_to_pil(images)

    return images
# https://www.pexels.com/photo/a-beagle-on-green-grass-field-8306128/

# Encode with VAE

## Inversion
@torch.no_grad()
def invert(
    start_latents,
    prompt,
    guidance_scale=7.5,
    num_inference_steps=80,
    num_images_per_prompt=1,
    do_classifier_free_guidance=True,
    negative_prompt="",
    device=device,
    pipe=None,
):

    # Encode prompt
    text_embeddings = pipe._encode_prompt(
        prompt, device, num_images_per_prompt, do_classifier_free_guidance, negative_prompt
    )

    # Latents are now the specified start latents
    latents = start_latents.clone()

    # We'll keep a list of the inverted latents as the process goes on
    intermediate_latents = []

    # Set num inference steps
    pipe.scheduler.set_timesteps(num_inference_steps, device=device)

    # Reversed timesteps <<<<<<<<<<<<<<<<<<<<
    timesteps = reversed(pipe.scheduler.timesteps)

    for i in tqdm(range(1, num_inference_steps), total=num_inference_steps - 1):

        # We'll skip the final iteration
        if i >= num_inference_steps - 1:
            continue

        t = timesteps[i]

        # Expand the latents if we are doing classifier free guidance
        latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
        latent_model_input = pipe.scheduler.scale_model_input(latent_model_input, t)

        # Predict the noise residual
        noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

        # Perform guidance
        if do_classifier_free_guidance:
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

        current_t = max(0, t.item() - (1000 // num_inference_steps))  # t
        next_t = t  # min(999, t.item() + (1000//num_inference_steps)) # t+1
        alpha_t = pipe.scheduler.alphas_cumprod[current_t]
        alpha_t_next = pipe.scheduler.alphas_cumprod[next_t]

        # Inverted update step (re-arranging the update step to get x(t) (new latents) as a function of x(t-1) (current latents)
        latents = (latents - (1 - alpha_t).sqrt() * noise_pred) * (alpha_t_next.sqrt() / alpha_t.sqrt()) + (
            1 - alpha_t_next
        ).sqrt() * noise_pred

        # Store
        intermediate_latents.append(latents)

    return torch.cat(intermediate_latents)


#   carpeta de imágenes
os.makedirs("generated_images", exist_ok=True)

# 3. FastAPI init
app = FastAPI()
app.mount("/files", StaticFiles(directory="generated_images"), name="files")



    
# ----------------------
# Inicializar el backend
# ----------------------
app = FastAPI()
nest_asyncio.apply()
app.mount(
    "/files",
    StaticFiles(directory="generated_images"),
    name="files"
)


class PromptRequest(BaseModel):
    prompts: List[str]  
    question: str       
    numbers: Optional[List[int]] = None 
       
num_inference_steps = 100
guidance_scale = 7.5
attn_res = (16, 16)
steps_to_save_attention_maps = list(range(num_inference_steps))
max_iter_to_alter = 50
refinement_steps = 100
scale_factor = 30
iterative_refinement_steps = [0, 10, 20]
do_smoothing = True
smoothing_sigma = 0.2
smoothing_kernel_size = 3
temperature = 0.5
softmax_normalize = False
softmax_normalize_attention_maps = False
add_previous_attention_maps = True
previous_attention_map_anchor_step = None
loss_fn = "ntxent"


# Args  get_model
args = dict(
    
    output_dir="output",
    eta=1.0,
    metrics_to_compute="ImageReward",
    prompt_path="",  
    model_name="stable-diffusion-v1-5",
)
fkd_args = dict(
    lmbda=8.0,
    num_particles=4,
    adaptive_resampling=True,
    resample_frequency=10,
    time_steps=50,
    potential_type='max',
    resampling_t_start=10,
    resampling_t_end=50,
    guidance_reward_fn='ImageReward',
    use_smc=True,
)

ns = {**args, **fkd_args}
all_args = argparse.Namespace(**ns)
all_args.num_inference_steps = fkd_args["time_steps"]



# 6. Endpoint
@app.post("/generate_custom")
async def generate_custom(req: PromptRequest):
    pipe = get_model(all_args.model_name)
    pipe = pipe.to("cuda")
    adapter_id = "latent-consistency/lcm-lora-sdv1-5"
    pipe.load_lora_weights(adapter_id)
    pipe.fuse_lora()
    guidance_scale = 0
    
    #Para Latent consistency en fk conform:
    args = dict(
    
    output_dir="output",
    eta=1.0,
    metrics_to_compute="ImageReward",
    prompt_path="",  
    model_name="stable-diffusion-v1-5",
)
    fkd_args = dict(
        lmbda=8.0,
        num_particles=4,
        adaptive_resampling=True,
        resample_frequency=10,
        time_steps=30,
        potential_type='max',
        resampling_t_start=10,
        resampling_t_end=50,
        guidance_reward_fn='ImageReward',
        use_smc=True,
    )

    ns = {**args, **fkd_args}
    all_args = argparse.Namespace(**ns)
    all_args.num_inference_steps = fkd_args["time_steps"]

    
    prompts = req.prompts[:3]
    question = req.question
    image_paths = []
    answers = []
    scenes = ["first scene, tell the story like if its a movie", "second scene, talk like if you are a pirate", "final scene, talk like a storyteller"]
    prompts = [
    {"prompt": prompts[0]},
    {"prompt": prompts[1]},
    {"prompt": prompts[2]},
    
]
    all_numbers = req.numbers
            
           
    token_groups=[all_numbers[0:2], all_numbers[2:4]]
    print(token_groups)
    token_groups2=[all_numbers[4:6], all_numbers[6:8]]
    token_groups3=[all_numbers[8:10], all_numbers[10:12]]
    
    
    for i, p in enumerate(prompts):
        prompt=[p['prompt']]*fkd_args['num_particles']
        if i==1:
            token_groups = token_groups2
        if i == 2:
            token_groups = token_groups3
        image = pipe(
            prompt=prompt,
            token_groups=token_groups,  
            fkd_args=fkd_args,
            num_inference_steps=fkd_args["time_steps"],
            eta=all_args.eta,
            guidance_scale=guidance_scale,
            
            max_iter_to_alter=max_iter_to_alter,
            attn_res=attn_res,
            scale_factor=scale_factor,
            iterative_refinement_steps=iterative_refinement_steps,
            steps_to_save_attention_maps=steps_to_save_attention_maps,
            do_smoothing=do_smoothing,
            smoothing_sigma=smoothing_sigma,
            smoothing_kernel_size=smoothing_kernel_size,
            temperature=temperature,
            refinement_steps=refinement_steps,
            softmax_normalize=softmax_normalize,
            softmax_normalize_attention_maps=softmax_normalize_attention_maps,
            add_previous_attention_maps=add_previous_attention_maps,
            previous_attention_map_anchor_step=previous_attention_map_anchor_step,
            loss_fn=loss_fn,
        )[0] 
        filename = f"generated_images/image_{i + 1}.png"  
        image[0].save(filename)
        image_paths.append(filename)
    del pipe   
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
        
    janus_path = "deepseek-ai/Janus-Pro-1B"
    vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(janus_path)
    tokenizer = vl_chat_processor.tokenizer
    vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(janus_path, trust_remote_code=True)
    vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()
    for i, img_path in enumerate(image_paths):
        question += scenes[i] + "\n"
        
        conversation = [
            {
                "role": "<|User|>",
                "content": f"<image_placeholder>\n{question}",
                "images": [img_path],   
            },
            {"role": "<|Assistant|>", "content": ""},
        ]
        
        pil_images = load_pil_images(conversation)

       
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=pil_images,
            force_batchify=True
        ).to(vl_gpt.device)
        inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

      
        output = vl_gpt.language_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=prepare_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=512,
            do_sample=False,
            use_cache=True,
        )[0]

        # Decodificamos y almacenamos
        text = tokenizer.decode(output.cpu().tolist(), skip_special_tokens=True)
        answers.append(text)

    return {
        "message": "Proceso completado",
        "generated_images": image_paths,
        "answers": answers     # lista de 3 strings
    }

@app.post("/generate_full")
async def generate_full(req: PromptRequest):
    
    
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    adapter_id = "latent-consistency/lcm-lora-sdxl"
    pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
    pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
    pipe.to("cuda")
    pipe.load_lora_weights(adapter_id)
    pipe.fuse_lora()

    prompts = req.prompts[:3]
    question = req.question
    image_paths = []
    answers = []
    scenes = ["first scene", "second scene", "final scene"]
   
    for i, p in enumerate(prompts[:3]):
        image = pipe(prompt=p, num_inference_steps=10, guidance_scale=0).images[0]
        filename = f"generated_images/image_{i + 5}.png"  
        image.save(filename)
        image_paths.append(filename)
        
        




       

        

        

    del pipe   
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    prompts = req.prompts[:3]
    question = req.question
    image_paths = []
    answers = []
    scenes = ["first scene, tell the story like if its a movie", "second scene, talk like if you are a pirate", "final scene, talk like a storyteller"]
    prompts = [
        {"prompt": prompts[0]},
        {"prompt": prompts[1]},
        {"prompt": prompts[2]},

    ]
    all_numbers = req.numbers


    token_groups=[all_numbers[0:2], all_numbers[2:4]]
    print(token_groups)
    token_groups2=[all_numbers[4:6], all_numbers[6:8]]
    token_groups3=[all_numbers[8:10], all_numbers[10:12]]
    pipe = get_model(all_args.model_name)
    pipe = pipe.to("cuda")
    for i, p in enumerate(prompts):
        
        prompt=[p['prompt']]*fkd_args['num_particles']
        if i==1:
            token_groups = token_groups2
        if i == 2:
            token_groups = token_groups3


        input_image = Image.open(f"generated_images/image_{i + 5}.png").convert("RGB")

    
        input_image = input_image.resize((512, 512))
        input_image_prompt = p['prompt']
        with torch.no_grad():

            input_tensor = tfms.functional.to_tensor(input_image).unsqueeze(0).to(device).to(torch.float16)

              # Asegúrate de escalar a [-1, 1] si eso espera tu VAE
            input_tensor = input_tensor * 2 - 1

              # Codificar usando el VAE
            latent = pipe.vae.encode(input_tensor)

            # Samplear y escalar latente
        l = 0.18215 * latent.latent_dist.sample()

            # Inversión (asumiendo que esta función está bien definida)
        inverted_latents = invert(l, input_image_prompt, num_inference_steps=50,pipe= pipe)    
        image = pipe(
                prompt=prompt,
                latents=inverted_latents[-(20 + 1)][None],
                token_groups=token_groups,  
                fkd_args=fkd_args,
                num_inference_steps=fkd_args["time_steps"],
                eta=all_args.eta,
                guidance_scale=guidance_scale,
                start_step=20,
                max_iter_to_alter=max_iter_to_alter,
                attn_res=attn_res,
                scale_factor=scale_factor,
                iterative_refinement_steps=iterative_refinement_steps,
                steps_to_save_attention_maps=steps_to_save_attention_maps,
                do_smoothing=do_smoothing,
                smoothing_sigma=smoothing_sigma,
                smoothing_kernel_size=smoothing_kernel_size,
                temperature=temperature,
                refinement_steps=refinement_steps,
                softmax_normalize=softmax_normalize,
                softmax_normalize_attention_maps=softmax_normalize_attention_maps,
                add_previous_attention_maps=add_previous_attention_maps,
                previous_attention_map_anchor_step=previous_attention_map_anchor_step,
                loss_fn=loss_fn,
            )[0] 
        filename = f"generated_images/image_{i + 1}.png"  
        image[0].save(filename)
        image_paths.append(filename)
    del pipe   
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    janus_path = "deepseek-ai/Janus-Pro-1B"
    vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(janus_path)
    tokenizer = vl_chat_processor.tokenizer
    vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(janus_path, trust_remote_code=True)
    vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()
    for i, img_path in enumerate(image_paths[:3]):
        question += scenes[i] + "\n"

        conversation = [
                {
                    "role": "<|User|>",
                    "content": f"<image_placeholder>\n{question}",
                    "images": [img_path],   
                },
                {"role": "<|Assistant|>", "content": ""},
            ]

        pil_images = load_pil_images(conversation)


        prepare_inputs = vl_chat_processor(
                conversations=conversation,
                images=pil_images,
                force_batchify=True
            ).to(vl_gpt.device)
        inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)


        output = vl_gpt.language_model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=prepare_inputs.attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=512,
                do_sample=False,
                use_cache=True,
            )[0]

            # Decodificamos y almacenamos
        text = tokenizer.decode(output.cpu().tolist(), skip_special_tokens=True)
        answers.append(text)

    return {
            "message": "Proceso completado",
            "generated_images": image_paths,
            "answers": answers     # lista de 3 strings
        }

# Modelo Janus


# Carpeta para guardar imágenes
os.makedirs("generated_images", exist_ok=True)

# ----------------------
# Endpoint principal
# ----------------------

@app.post("/generate_and_answer")
async def generate_and_answer(req: PromptRequest):
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    adapter_id = "latent-consistency/lcm-lora-sdxl"
    pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
    pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
    pipe.to("cuda")
    pipe.load_lora_weights(adapter_id)
    pipe.fuse_lora()

    prompts = req.prompts[:3]
    question = req.question
    image_paths = []
    answers = []
    scenes = ["first scene", "second scene", "final scene"]
   
    for i, p in enumerate(prompts[:3]):
        image = pipe(prompt=p, num_inference_steps=4, guidance_scale=0).images[0]
        filename = f"generated_images/image_{i + 1}.png"  
        image.save(filename)
        image_paths.append(filename)
        
    del pipe   
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    janus_path = "deepseek-ai/Janus-Pro-1B"
    vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(janus_path)
    tokenizer = vl_chat_processor.tokenizer
    vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(janus_path, trust_remote_code=True)
    vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()
    
    for i, img_path in enumerate(image_paths):
        question += scenes[i] + "\n"
        
        conversation = [
            {
                "role": "<|User|>",
                "content": f"<image_placeholder>\n{question}",
                "images": [img_path],   # sólo la ruta
            },
            {"role": "<|Assistant|>", "content": ""},
        ]
        
        pil_images = load_pil_images(conversation)

        
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=pil_images,
            force_batchify=True
        ).to(vl_gpt.device)
        inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

        
        output = vl_gpt.language_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=prepare_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=512,
            do_sample=False,
            use_cache=True,
        )[0]

        
        text = tokenizer.decode(output.cpu().tolist(), skip_special_tokens=True)
        answers.append(text)

    return {
        "message": "Proceso completado",
        "generated_images": image_paths,
        "answers": answers     
    }


# ----------------------
# Iniciar ngrok y servidor
# ----------------------
public_url = ngrok.connect(8000)
print("Servidor público:", public_url)
uvicorn.run(app, port=8000)


2025-06-29 23:49:10.157057: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/safernandez/.local/lib/python3.8/site-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


Servidor público: NgrokTunnel: "https://08c8-193-147-26-250.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1526632]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

[[1, 1], [2, 2]]
Args: {'lmbda': 8.0, 'num_particles': 4, 'adaptive_resampling': True, 'resample_frequency': 10, 'time_steps': 50, 'potential_type': 'max', 'resampling_t_start': 10, 'resampling_t_end': 50, 'guidance_reward_fn': 'ImageReward', 'use_smc': True}


...

`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
load checkpoint from /home/safernandez/.cache/ImageReward/ImageReward.pt
checkpoint loaded
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
Args: {'lmbda': 8.0, 'num_particles': 4, 'adaptive_resampling': True, 'resample_frequency': 10, 'time_steps': 50, 'potential_type': 'max', 'resampling_t_start': 10, 'resampling_t_end': 50, 'guidance_reward_fn': 'ImageReward', 'use_smc': True}


  0%|          | 0/50 [00:00<?, ?it/s]

`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
Args: {'lmbda': 8.0, 'num_particles': 4, 'adaptive_resampling': True, 'resample_frequency': 10, 'time_steps': 50, 'potential_type': 'max', 'resampling_t_start': 10, 'resampling_t_end': 50, 'guidance_reward_fn': 'ImageReward', 'use_smc': True}


  0%|          | 0/50 [00:00<?, ?it/s]

`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
`metric_to_chase` will be ignored as it only applies to 'LLMGrader' as the `reward_name`
INFO:     94.76.151.213:0 - "POST /generate_custom HTTP/1.1" 200 OK
INFO:     94.76.151.213:0 - "GET /files/image_1.png HTTP/1.1" 200 OK
INFO:     94.76.151.213:0 - "GET /files/image_2.png HTTP/1.1" 200 OK
INFO:     94.76.151.213:0 - "GET /files/image_3.png HTTP/1.1" 200 OK
